# Battery Sizing Module 

In [ ]:
import math

In [1]:
### Processing Rainfall Cycle Data: into size metrics
def get_battery_size(df, minimum_voltage, EOL, red_factor_th, red_factor_p):
    size_metrics = {}
    
    ## Required Power
    size_metrics['max_disch_MW'] =red_factor_p*max(df['max_disch_MW'])
    size_metrics['max_ch_MW'] = red_factor_p*min(df['max_ch_MW'])
    
    ## Required Capacity (Throughput)
    et = 0 # running value
    et_list =[]
    for row in df.iterrows():
        row = row[1]
        et += float(row['disch_th_kWh']+row['ch_th_kWh'])
        et_list.append(et)
        
    et_list = [max([abs(ite) for ite in et_list])]    
    et_list.append(abs(max(df['disch_th_kWh']))) # capacity required due to discharge
    et_list.append(abs(min(df['ch_th_kWh']))) # capacity required due to charge
    size_metrics['th_kWh'] = max(et_list)
    size_metrics['th_kWh'] = size_metrics['th_kWh']*(100/EOL) # capacity is required at EOL
    size_metrics['th_kWh'] = size_metrics['th_kWh']*red_factor_th
    
    ## Required Voltage
    size_metrics['min_v'] = minimum_voltage
    
    return size_metrics

In [6]:
### Getting Battery Design from processed size metrics
def get_battery_design(size_metrics, cf):
    
    # Cell Model Variables
    chem_nom_voltage = cf['Nom Voltage_V'] #V
    chem_nom_cap = cf['Capacity_Ah'] #Ah
    chem_max_cont_current = cf['Max Continuous Current_A']
    
    chem_nom_energy = chem_nom_cap * chem_nom_voltage
    chem_nom_power = chem_max_cont_current*chem_nom_voltage
    
    design_metrics={}
    ##Cells in series: min voltage
    design_metrics["tot_cells_series"] = math.ceil(size_metrics['min_v']/chem_nom_voltage)
    
    ##Cells in parallel: power & capacity (dependant on the limiting factor)
    design_metrics['max_disch_power'] = size_metrics['max_disch_MW']
    design_metrics['max_power']= max(abs(size_metrics['max_disch_MW']), abs(size_metrics['max_ch_MW']))
    ## Determining parallel strings based on power or capacity
    c_p_power = math.ceil(design_metrics['max_power']/(chem_nom_power*design_metrics["tot_cells_series"]))
    c_p_cap = math.ceil(math.ceil((size_metrics['th_kWh']*1000)/chem_nom_energy)/design_metrics["tot_cells_series"])
    
    design_metrics["tot_cells_parallel"] = max(c_p_power,c_p_cap)
    design_metrics["tot_cells"] =design_metrics["tot_cells_series"]*design_metrics["tot_cells_parallel"]
    design_metrics["tot_cap"] = (chem_nom_energy*design_metrics["tot_cells"])/1000 ## kWh
    
    return design_metrics